In [43]:
import numpy as np
import tensorflow as tf
import random as rn
import os
import matplotlib.pyplot as plt
%matplotlib inline
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(1)
rn.seed(1)
from keras import backend as K
tf.set_random_seed(1)
sess = tf.Session(graph=tf.get_default_graph())
K.set_session(sess)
import sys
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.layers import Conv1D,MaxPooling1D,Conv2D,MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TerminateOnNaN
import keras.regularizers
import scipy
import math
import sys
import pandas as pd
from scipy.ndimage.filters import gaussian_filter1d
from sklearn.metrics import mean_squared_error
from scipy.stats import linregress
from scipy import interpolate
from scipy import signal
#import cPickle as pickle
#from video_process_utils import *
import collections
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from PIL import Image
import matplotlib.pylab as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="0" 

In [44]:
# Pull down GDI list and Vide List files

In [45]:
#gdi_df = pd.read_csv('./data/test/labels/gdi.csv')
gdi_df = pd.read_csv('./data/gdi.csv')
print(gdi_df.head())
print(gdi_df.shape)
#video_df = pd.read_csv('./data/test/labels/video_list.csv')
video_df = pd.read_csv('./data/video_list.csv')
print(video_df.shape)
print(video_df.shape)

   examid  Patient_ID side        GDI
0      32        3364    L  79.507851
1      32        3364    R  87.923384
2      35        3365    L  80.504580
3      35        3365    R  75.721122
4      36        3367    L  49.978819
(16723, 4)
(13841, 3)
(13841, 3)


In [46]:
# Merge GDI list and Video List

merged_df = pd.merge(gdi_df, video_df, on=['examid'], how='outer')
print(merged_df.head())
print(merged_df.shape)


   examid  Patient_ID side        GDI Event_Date  \
0      32      3364.0    L  79.507851  24-Oct-94   
1      32      3364.0    R  87.923384  24-Oct-94   
2      35      3365.0    L  80.504580  07-Mar-96   
3      35      3365.0    R  75.721122  07-Mar-96   
4      36      3367.0    L  49.978819  31-Jul-97   

                                          Video_File  
0  \\fapvids\Videos\GILLETTEVIEWER\1994\06988401.mp4  
1  \\fapvids\Videos\GILLETTEVIEWER\1994\06988401.mp4  
2  \\fapvids\Videos\GILLETTEVIEWER\1996\07022001.mp4  
3  \\fapvids\Videos\GILLETTEVIEWER\1996\07022001.mp4  
4  \\fapvids\Videos\GILLETTEVIEWER\1997\07337701.mp4  
(22228, 6)


In [47]:
gdi_exam_grp = gdi_df.groupby(gdi_df['examid'])
#print(gdi_exam_grp.head())

In [48]:
merged_gdiavg_df = pd.merge(gdi_exam_grp.mean(), video_df, on=['examid'], how='outer')
#merged_gdiavg_df.to_csv('./data/test/labels/merged_gdiavg.csv')
merged_gdiavg_df.to_csv('./data/merged_gdiavg.csv')

print(merged_gdiavg_df.shape)
print(merged_gdiavg_df.head())

(13960, 5)
   examid  Patient_ID        GDI Event_Date  \
0      32      3364.0  83.715617  24-Oct-94   
1      35      3365.0  78.112851  07-Mar-96   
2      36      3367.0  53.457512  31-Jul-97   
3      37      3369.0  60.827350  22-Aug-95   
4      40      3371.0  63.043201  20-Dec-94   

                                          Video_File  
0  \\fapvids\Videos\GILLETTEVIEWER\1994\06988401.mp4  
1  \\fapvids\Videos\GILLETTEVIEWER\1996\07022001.mp4  
2  \\fapvids\Videos\GILLETTEVIEWER\1997\07337701.mp4  
3  \\fapvids\Videos\GILLETTEVIEWER\1995\07399701.mp4  
4  \\fapvids\Videos\GILLETTEVIEWER\1994\07720401.mp4  


In [49]:
# Remove videos without a GDI

In [50]:
d_mergedgdiavg_new_df = merged_gdiavg_df[True != merged_gdiavg_df['GDI'].isnull()]

In [51]:
print(d_mergedgdiavg_new_df.shape)
print(d_mergedgdiavg_new_df.head())

(8456, 5)
   examid  Patient_ID        GDI Event_Date  \
0      32      3364.0  83.715617  24-Oct-94   
1      35      3365.0  78.112851  07-Mar-96   
2      36      3367.0  53.457512  31-Jul-97   
3      37      3369.0  60.827350  22-Aug-95   
4      40      3371.0  63.043201  20-Dec-94   

                                          Video_File  
0  \\fapvids\Videos\GILLETTEVIEWER\1994\06988401.mp4  
1  \\fapvids\Videos\GILLETTEVIEWER\1996\07022001.mp4  
2  \\fapvids\Videos\GILLETTEVIEWER\1997\07337701.mp4  
3  \\fapvids\Videos\GILLETTEVIEWER\1995\07399701.mp4  
4  \\fapvids\Videos\GILLETTEVIEWER\1994\07720401.mp4  


In [52]:
print(merged_gdiavg_df['Video_File'].str.contains('10366507', na=False).any())
print((d_mergedgdiavg_new_df['Video_File'].str.contains('10366507', na=False).any()))

True
False


In [54]:
# THIS IS THE SCRIPT TO GENERATE THE MAPPING FROM IMAGES TO FILE PATHS TO LOAD TRAINING EXAMPLES

image_file_path2 = []
labels2 = []
for root, dirs, files in os.walk('./data/densepose-out'):
    #print(dirs)
    if(root.endswith('processed')):
        #image_dir_name = (root[root.rfind('\\')+1:])
        image_dir_name = (root[root.rfind('\\')+1:])[:8] #ASSUME 8 DIGITS ONLY
#         print(image_dir_name)
#          break
        if(d_mergedgdiavg_new_df['Video_File'].str.contains(image_dir_name, na=False).any()):
            img_cnt = 0
            for file in files:
                if(file.endswith('_IUV.png')):
                    image_file_path2.append(os.path.join(root, file))
                    labels2.append(d_mergedgdiavg_new_df.GDI[d_mergedgdiavg_new_df['Video_File'].str.contains(image_dir_name, na=False)].values[0])
                    img_cnt+=1
#                     if(img_cnt>10):
#                         break;
#         else:
#             print(image_dir_name) # print all folders that are lost to our joins and exceptions

In [55]:
print(len(image_file_path2))

68931


In [ ]:
# image_file_path2 = []
# labels2 = []
# for root, dirs, files in os.walk('./data/densepose-out'):
#     #print(dirs)
#     if(root.endswith('processed')):
#         #image_dir_name = (root[root.rfind('\\')+1:])
#         image_dir_name = (root[root.rfind('\\')+1:])[:8] #ASSUME 8 DIGITS ONLY
# #         print(image_dir_name)
# #          break
#         if(d_mergedgdiavg_new_df['Video_File'].str.contains(image_dir_name, na=False).any()):
#             img_cnt = 0
#             for file in files:
#                 if(file.endswith('_IUV.png')):
#                     image_file_path2.append(os.path.join(root, file))
#                     labels2.append(d_mergedgdiavg_new_df.GDI[d_mergedgdiavg_new_df['Video_File'].str.contains(image_dir_name, na=False)].values[0])
#                     img_cnt+=1
# #                     if(img_cnt>10):
# #                         break;

,Unnamed: 0,examid,Patient_ID,GDI,Event_Date,Video_File
0,0,32,3364.0,83.715617,1994-10-24,\\fapvids\Videos\GILLETTEVIEWER\1994\06988401.mp4
1,1,35,3365.0,78.112851,1996-03-07,\\fapvids\Videos\GILLETTEVIEWER\1996\07022001.mp4
2,2,36,3367.0,53.457512,1997-07-31,\\fapvids\Videos\GILLETTEVIEWER\1997\07337701.mp4
3,3,37,3369.0,60.827350,1995-08-22,\\fapvids\Videos\GILLETTEVIEWER\1995\07399701.mp4
4,4,40,3371.0,63.043201,1994-12-20,\\fapvids\Videos\GILLETTEVIEWER\1994\07720401.mp4


In [7]:
d_mergedgdiavg_new_df = mergedgdiavg_new_df[True != mergedgdiavg_new_df['GDI'].isnull()]

In [8]:
image_path=d_mergedgdiavg_new_df['Video_File']
image_path.head()

0    \\fapvids\Videos\GILLETTEVIEWER\1994\06988401.mp4
1    \\fapvids\Videos\GILLETTEVIEWER\1996\07022001.mp4
2    \\fapvids\Videos\GILLETTEVIEWER\1997\07337701.mp4
3    \\fapvids\Videos\GILLETTEVIEWER\1995\07399701.mp4
4    \\fapvids\Videos\GILLETTEVIEWER\1994\07720401.mp4
Name: Video_File, dtype: object

In [9]:
labels=d_mergedgdiavg_new_df['GDI']
labels.head()

0    83.715617
1    78.112851
2    53.457512
3    60.827350
4    63.043201
Name: GDI, dtype: float64

In [17]:
image_file_path = []
labels = []
for root, dirs, files in os.walk('./data/densepose-out'):
    if(root.endswith('processed')):
        image_dir_name = (root[root.rfind('\\')+1:])
        if(d_mergedgdiavg_new_df['Video_File'].str.contains(image_dir_name, na=False).any()):
            dirImages = []
            img_cnt = 0
            for file in files:
                if(file.endswith('_IUV.png')):
                    image_file_path.append(os.path.join(root, file))
                    labels.append(d_mergedgdiavg_new_df.GDI[d_mergedgdiavg_new_df['Video_File'].str.contains(image_dir_name, na=False)].values[0])

In [18]:
len(image_file_path)

234033

In [19]:
len(labels)

234033

In [56]:
image_path_series=pd.Series(image_file_path2)
labels_series=pd.Series(labels2)
imgaepath_gdi_df = pd.DataFrame()
imgaepath_gdi_df['image_path']=image_path_series.values
imgaepath_gdi_df['labels']=labels_series.values
imgaepath_gdi_df.head()

,image_path,labels
0,./data/densepose-out\10576905-processed\000053...,62.141802
1,./data/densepose-out\10576905-processed\000054...,62.141802
2,./data/densepose-out\10576905-processed\000055...,62.141802
3,./data/densepose-out\10576905-processed\000056...,62.141802
4,./data/densepose-out\10576905-processed\000057...,62.141802


In [58]:
imgaepath_gdi_df.to_csv('./data/imagepath_gdi_allframes_205.csv')